 En esta parte del trabajo solicitado se crearan las funciones para los endpoints 
 que se consumiran en la API

Se importaron las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Se trabajo con el dataset ya arreglado y organizado

In [2]:
df=pd.read_csv("df_arreglado.csv")

In [9]:
df

,Unnamed: 0,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,release_year,return
0,0,Toy Story Collection,30000000.0,"['Animation', 'Comedy', 'Family']",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,['English'],Released,NaN,Toy Story,7.7,1995,12.451801
1,1,NaN,65000000.0,"['Adventure', 'Fantasy', 'Family']",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],1995-12-15,262797249.0,104.0,"['English', 'Français']",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,1995,4.043035
2,2,Grumpy Old Men Collection,0.0,"['Romance', 'Comedy']",15602,en,A family wedding reignites the ancient feud be...,11.712900,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],1995-12-22,0.0,101.0,['English'],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,1995,0.000000
3,3,NaN,16000000.0,"['Comedy', 'Drama', 'Romance']",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,['Twentieth Century Fox Film Corporation'],['United States of America'],1995-12-22,81452156.0,127.0,['English'],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,1995,5.090760
4,4,Father of the Bride Collection,0.0,['Comedy'],11862,en,Just when George Banks has recovered from his ...,8.387519,"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],1995-02-10,76578911.0,106.0,['English'],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,1995,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45371,45460,NaN,0.0,"['Drama', 'Action', 'Romance']",30840,en,"Yet another version of the classic epic, with ...",5.683753,"['Westdeutscher Rundfunk (WDR)', 'Working Titl...","['Canada', 'Germany', 'United Kingdom', 'Unite...",1991-05-13,0.0,104.0,['English'],Released,NaN,Robin Hood,5.7,1991,0.000000
45372,45462,NaN,0.0,['Drama'],111109,tl,An artist struggles to finish his work while a...,0.178241,['Sine Olivia'],['Philippines'],2011-11-17,0.0,360.0,[''],Released,NaN,Century of Birthing,9.0,2011,0.000000
45373,45463,NaN,0.0,"['Action', 'Drama', 'Thriller']",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,['American World Pictures'],['United States of America'],2003-08-01,0.0,90.0,['English'],Released,A deadly game of wits.,Betrayal,3.8,2003,0.000000
45374,45464,NaN,0.0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,['Yermoliev'],['Russia'],1917-10-21,0.0,87.0,NaN,Released,NaN,Satan Triumphant,0.0,1917,0.000000


API-1

Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes 

In [20]:
def peliculas_mes(mes: str) -> dict:
    months_translated= {
    'enero': 'January',
    'febrero': 'February',
    'marzo': 'March',
    'abril': 'April',
    'mayo': 'May',
    'junio': 'June',
    'julio': 'July',
    'agosto': 'August',
    'septiembre': 'September',
    'octubre': 'October',
    'noviembre': 'November',
    'diciembre': 'December'}  
    fechas = pd.to_datetime(df['release_date'], format= '%Y-%m-%d')
    n_mes = fechas[fechas.dt.strftime('%B').str.capitalize() == months_translated[str(mes).lower()]]
    respuesta = n_mes.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

In [19]:
peliculas_mes('diciembre')

{'mes': 'diciembre', 'cantidad': 3786}

Esta es la primera funcion utilizada pero da error al correr en FastAPI

In [4]:
df.release_date=pd.to_datetime(df['release_date'], format='%Y-%m-%d',errors='coerce')
meses=df.release_date.dt.month
m1=pd.DataFrame(meses.value_counts())
m1.reset_index(inplace=True)
m1

,index,release_date
0,1,5912
1,9,4838
2,10,4615
3,12,3786
4,11,3661
5,3,3553
6,4,3453
7,8,3394
8,5,3339
9,6,3153


In [5]:
def peliculas_mes(mes):
    '''se ingresa el  mes y la función retorna la cantidad de peliculas que se estrenaron
    ese mes historicamente'''
    mes=mes.lower()
    
    m2={
        'enero':1,
        'febrero':2,
        'marzo':3,
        'abril':4,
        'mayo':5,
        'junio':6,
        'julio':7,
        'agosto':8,
        'septiembre':9,
        'octubre':10,
        'noviembre':11,
        'diciembre':12
    }
    est=m1.release_date[m1['index']==m2[mes]]
    est=est.tolist()
    respuesta=est[0]
    return{'mes':mes,'cantidad':respuesta}

In [6]:
peliculas_mes("Diciembre")

{'mes': 'diciembre', 'cantidad': 3786}

API-2

Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrenaron ese dia 

In [7]:
def peliculas_dia(dia:str):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se
    estrenaron ese dia historicamente'''
    fechas=pd.to_datetime(df['release_date'],format='%Y-%m-%d')
    ndia = fechas[fechas.dt.day_name(locale='es_AR') == dia.capitalize()]
    respuesta=ndia.shape[0]
    return{'dia':dia, 'cantidad':respuesta}

In [8]:
peliculas_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

API-3

Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio

In [9]:
df.belongs_to_collection=df.belongs_to_collection.str.lower()
m2=df[["belongs_to_collection","revenue"]]

In [10]:
m2

,belongs_to_collection,revenue
0,toy story collection,373554033.0
1,NaN,262797249.0
2,grumpy old men collection,0.0
3,NaN,81452156.0
4,father of the bride collection,76578911.0
...,...,...
45371,NaN,0.0
45372,NaN,0.0
45373,NaN,0.0
45374,NaN,0.0


In [11]:
def franquicia(franquicia:str):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio'''
    if isinstance(franquicia,str):
        franquicia=franquicia.lower()
        franquicia=unicodedata.normalize('NFKD',franquicia).encode(
            'ascii','ignore').decode('utf-8','ignore')
        
        ganancias=m2['revenue'][m2['belongs_to_collection'].str.contains(franquicia)==True]
        respuesta1=ganancias.shape[0]
        respuesta2=ganancias.sum()
        respuesta3=ganancias.mean()
    
    return{'franquicias':franquicia,'cantidad':respuesta1,'ganancia_total':respuesta2,'ganancia_promedio':respuesta3}

In [12]:
franquicia('Toy Story Collection')

{'franquicias': 'toy story collection',
 'cantidad': 3,
 'ganancia_total': 1937890605.0,
 'ganancia_promedio': 645963535.0}

API-4

Ingresa el pais, retornando la cantidad de peliculas producidas en el mismo

In [10]:
m1=df[["title", "production_countries"]]
m1


,title,production_countries
0,Toy Story,['United States of America']
1,Jumanji,['United States of America']
2,Grumpier Old Men,['United States of America']
3,Waiting to Exhale,['United States of America']
4,Father of the Bride Part II,['United States of America']
...,...,...
45371,Robin Hood,"['Canada', 'Germany', 'United Kingdom', 'Unite..."
45372,Century of Birthing,['Philippines']
45373,Betrayal,['United States of America']
45374,Satan Triumphant,['Russia']


In [11]:
def peliculas_pais(pais):

    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo'''
    if isinstance(pais, str):
        pais=pais.lower()
        pais=unicodedata.normalize('NFKD', pais).encode(
            'ascii','ignore').decode('utf-8','ignore')
        cantidad=m1['title']
        [m1['production_countries'].str.contains(pais)==True]
        cantidad=df['production_countries'].apply(lambda x :str(x).lower()).map(str.lower).apply(lambda x:pais in x)
        respuesta= cantidad.shape [0]
        return{'pais': pais, 'cantidad': respuesta}


In [12]:
peliculas_pais('united states of america')

{'pais': 'united states of america', 'cantidad': 45376}

API-5

Ingresa la productora, retornando la ganancia total y la cantidad de peliculas que produjeron

In [16]:
def productoras(productora:str):
    '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron'''
    prod=df[['production_companies','budget','revenue']].dropna()
    prod['production_companies']=prod['production_companies'].map(str.lower)
    prod=prod[prod.production_companies.str.contains(productora.lower(),regex=False)]
    cantidad=prod.shape[0]
    gan_tot=(prod['revenue']-prod['budget']).sum()
    return{'productora': productora, 'ganancia_total':gan_tot, 'cantidad':cantidad}


In [17]:
productoras('Warner Bros.')

{'productora': 'Warner Bros.',
 'ganancia_total': 42271724104.0,
 'cantidad': 1374}

API-6

Ingresa la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo

In [16]:
def retorno(pelicula: str) -> dict:
    pelicula_df = df.loc[df['title'] == pelicula.title()]
    inversion = pelicula_df['budget'].iloc[0].item()
    ganancia = pelicula_df['revenue'].iloc[0].item()
    retorno= pelicula_df['return'].iloc[0].item()
    anio = pelicula_df['release_year'].iloc[0].item()
    return {'pelicula': pelicula, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio }

In [17]:
retorno ('toy story')

{'pelicula': 'toy story',
 'inversion': 30000000.0,
 'ganancia': 373554033.0,
 'retorno': 12.4518011,
 'anio': 1995}

ML
Ingresa un nombre de pelicula y recomienda las similares en una lista
    

In [3]:
df=pd.read_csv('df_arreglado.csv')

Se genera un dataset llamado copia_df ya que la capacidad de la memoria no permite 
usar el de 45376 registros, solo se hara un sample para utilizar el modelo elegido

In [4]:
copia_df=df.copy()

In [5]:
copia_df = copia_df.drop(df.index[:40376])

Con el Vectorizer se eliminaron las palabras no utiles a la busqueda y ademas se reemplazaron los valores nulos

In [6]:
tfidf=TfidfVectorizer(stop_words='english')
copia_df['overview']=copia_df['overview'].fillna('')
tfidf_matrix=tfidf.fit_transform(copia_df['overview'])
tfidf_matrix.shape

(5000, 24367)

Se uso  la matriz de similitud de coseno para comprender la columna de descripción general e identificar patrones y estructuras similares a partir de ella, importando el kernel lineal. 

In [7]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [8]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

Ahora, en este punto, se genero la funcion de recomendacion

In [19]:
def recomendacion(titulo):
    idx = indices[titulo]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    recommendations=list(copia_df['title'].iloc[movie_indices].str.title())
    return {'lista recomendada': recommendations}
    

In [20]:
recomendacion('Toy Story')

{'lista recomendada': ['You Are My Sunshine',
  'Office',
  'Tinker Ticker',
  'The Who: Thirty Years Of Maximum R&B',
  "Jessica Darling'S It List"]}